In [1]:
import requests
websiteURL = requests.get('https://en.wikipedia.org/wiki/The_Return_of_Superman_(TV_series)').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(websiteURL, 'lxml')
print(soup.prettify()) # check content of soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   The Return of Superman (TV series) - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"The_Return_of_Superman_(TV_series)","wgTitle":"The Return of Superman (TV series)","wgCurRevisionId":924169926,"wgRevisionId":924169926,"wgArticleId":42673797,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using Timeline","Webarchive template wayback links","CS1 uses Korean-language script (ko)","CS1 Korean-language sources (ko)","CS1 maint: archived copy as title","Articles with short description","Use mdy dates from May 2014","Articles containing Korean-language text","Articles with Korean-language external links","Korean Broadcasting System television programmes","South Korean travel televis

In [3]:
listOfTables = soup.findAll('table', {'class':'wikitable plainrowheaders'})
listOfTables

[<table class="wikitable plainrowheaders" style="width:73%;">
 <tbody><tr>
 <th scope="col" style="background:#c40233; color:#fff; width:14%;">Duration
 </th>
 <th scope="col" style="background:#c40233; color:#fff; width:8%;">Dad
 </th>
 <th scope="col" style="background:#c40233; color:#fff; width:24%;">Children
 </th>
 <th scope="col" style="background:#c40233; color:#fff; width:6%;">Mom/Wife
 </th>
 <th scope="col" style="background:#c40233; color:#fff; width:12%;">Filming location
 </th></tr>
 <tr>
 <td rowspan="2"><small>2016–<br/>Ep 154–Present</small>
 </td>
 <td rowspan="2"><a href="/wiki/Sam_Hammington" title="Sam Hammington">Sam Hammington</a><br/><small>Television Personality</small>
 </td>
 <td>Son: William Hammington (Jung Tae-oh) <br/><small>DOB: <span style="display:none"> (<span class="bday">2016-07-12</span>) </span>July 12, 2016<span class="noprint ForceAgeToShow"> (age 3)</span></small>
 </td>
 <td rowspan="2">Jung Yu-mi <br/><small>Housewife, Former Cafe Owner</small

In [4]:
len(listOfTables)

3

In [5]:
listOfTables[0]

<table class="wikitable plainrowheaders" style="width:73%;">
<tbody><tr>
<th scope="col" style="background:#c40233; color:#fff; width:14%;">Duration
</th>
<th scope="col" style="background:#c40233; color:#fff; width:8%;">Dad
</th>
<th scope="col" style="background:#c40233; color:#fff; width:24%;">Children
</th>
<th scope="col" style="background:#c40233; color:#fff; width:6%;">Mom/Wife
</th>
<th scope="col" style="background:#c40233; color:#fff; width:12%;">Filming location
</th></tr>
<tr>
<td rowspan="2"><small>2016–<br/>Ep 154–Present</small>
</td>
<td rowspan="2"><a href="/wiki/Sam_Hammington" title="Sam Hammington">Sam Hammington</a><br/><small>Television Personality</small>
</td>
<td>Son: William Hammington (Jung Tae-oh) <br/><small>DOB: <span style="display:none"> (<span class="bday">2016-07-12</span>) </span>July 12, 2016<span class="noprint ForceAgeToShow"> (age 3)</span></small>
</td>
<td rowspan="2">Jung Yu-mi <br/><small>Housewife, Former Cafe Owner</small>
</td>
<td rowspan=

In [6]:
from bs4 import Tag, NavigableString
def getText(obj):
    itemToAppend = 0
    if isinstance(obj.find("br").previous_sibling, NavigableString):
        itemToAppend = (str(obj.find("br").previous_sibling).strip())
    else:
        itemToAppend = (obj.find("br").previous_sibling.get_text().strip())
        if obj.find("br").previous_sibling.previous_sibling:
            itemToAppend =  str(obj.find("br").previous_sibling.previous_sibling).strip() + " " + itemToAppend
    return itemToAppend

In [7]:
def getEpisode(obj):
    itemToAppend = ""
    x = obj.findAll("br")
    temp_epList = []
    if x: 
        for item in x:
            if item.next_sibling:
                temp_epList.append(str(item.next_sibling).strip())
        itemToAppend = " ".join(temp_epList)
    else:
        p = obj.find("p")
        itemToAppend = p.get_text().strip()
    
    return itemToAppend

In [8]:
import pandas as pd
from bs4 import Tag, NavigableString
import re

def getDataFrame(table):
    trows = table.find_all('tr')
    
    episode = []
    father = []
    fatherJob = []
    child = []
    childBday = []
    mother = []
    motherJob = []
    filmLocation = []

    narratorCtr = 0
    nEpisode = []
    narrator = []

    for item in trows:
        row = item.find_all('td')
        if narratorCtr == 2:
            nEpisode.append(str(row[0].find("br").next_sibling).strip())
            narrator.append(row[1].find("a").get_text().strip())
            continue


        if len(row) == 0:
            narratorCtr = narratorCtr + 1
        else:
            ctr = 0

            if len(row) == 5:

                episode.append(getEpisode(row[0]))

                if row[1].find("a"):
                    father.append(row[1].find("a").get_text().strip())
                else:
                    father.append(getText(row[1]))

                fatherJob.append(row[1].find("small").get_text().strip())
                
                child.append(getText(row[2]))

                if row[2].find("span", {'class':'bday'}):
                    childBday.append(row[2].find("span", {'class':'bday'}).get_text())
                else:
                    childBday.append(row[2].find("small").get_text().strip())

                if row[3].find("a") and not bool(re.search(r'\d', row[3].find("a").get_text())): # arbitrary
                    mother.append(row[3].find("a").get_text().strip())
                else:
                    mother.append(getText(row[3]))
                motherJob.append(row[3].find("small").get_text().strip())

                filmLocation.append(row[4].get_text().strip())
            else:
                index = len(episode)-1
                episode.append(episode[index])
                father.append(father[index])
                fatherJob.append(fatherJob[index])
                mother.append(mother[index])
                motherJob.append(motherJob[index])
                filmLocation.append(filmLocation[index])
                
                child.append(getText(row[0]))

                if row[0].find("span", {'class':'bday'}):
                    childBday.append(row[0].find("span", {'class':'bday'}).get_text())
                else:
                    childBday.append(row[0].find("small").get_text().strip())
    dict_cast = {'episode':episode,
             'father': father, 
            'fatherJob': fatherJob,
            'child': child,
            'childBday': childBday,
            'mother': mother,
            'motherJob':motherJob,
            'filmLocation': filmLocation}
    cast_df = pd.DataFrame(dict_cast)
    
    dict_narrator = {'episode': nEpisode,
             'narrator': narrator}
    narrator_df = pd.DataFrame(dict_narrator)
    
    return cast_df, narrator_df

In [9]:
currentCast_df, currentNarrator_df = getDataFrame(listOfTables[0])
formerCast_df, formerNarrator_df = getDataFrame(listOfTables[1])
specialCast_df, _ = getDataFrame(listOfTables[2])

In [10]:
currentCast_df

,episode,father,fatherJob,child,childBday,mother,motherJob,filmLocation
0,Ep 154–Present,Sam Hammington,Television Personality,Son: William Hammington (Jung Tae-oh),2016-07-12,Jung Yu-mi,"Housewife, Former Cafe Owner","Eunpyung-gu, Seoul, South Korea"
1,Ep 154–Present,Sam Hammington,Television Personality,Son: Bentley Hammington (Jung Woo-sung),2017-11-08,Jung Yu-mi,"Housewife, Former Cafe Owner","Eunpyung-gu, Seoul, South Korea"
2,Ep 239–Present,Park Joo-ho,Professional Football (Soccer) player,Daughter: Park Na-eun (Eden),2015-05-11,Anna Park,Housewife,"Ulsan, South Korea"
3,Ep 239–Present,Park Joo-ho,Professional Football (Soccer) player,Son: Park Geon-hoo (Aciel),2017-08-21,Anna Park,Housewife,"Ulsan, South Korea"
4,Ep 289–present,Moon Hee-joon,Television Personality/Former Idol-Singer,Daughter: Moon Hee-yul (Jam-jam),2017-05-12,Soyul,Former Idol-singer,"Seoul, South Korea"
5,"Ep 195, 273–274, 279, 283 (special appearances...",Hong Kyung-min,Singer/Actor,Daughter: Hong Ra-won,2016-04-21,Kim Yuna,Haegeum Musician,"Seoul, South Korea"
6,"Ep 195, 273–274, 279, 283 (special appearances...",Hong Kyung-min,Singer/Actor,Daughter: Hong Ra-im,2019-01-01,Kim Yuna,Haegeum Musician,"Seoul, South Korea"


In [11]:
currentNarrator_df

,episode,narrator
0,"Episode 253–255, 258–present",Do Kyung-wan
1,Episode 273–present,Han Chae-ah


In [12]:
formerCast_df

,episode,father,fatherJob,child,childBday,mother,motherJob,filmLocation
0,Chuseok Special Part 1–3,Lee Hyun-woo,Singer/Actor,Son: Lee Dong-ha,DOB: September 2009,Yi Je-ni,Museum Curator,"Seoul, South Korea"
1,Chuseok Special Part 1–3,Lee Hyun-woo,Singer/Actor,Son: Lee Ju-ha,DOB: April 2011,Yi Je-ni,Museum Curator,"Seoul, South Korea"
2,Ep 25–31,Kim Jung-tae,Actor,Son: Kim Ji-hoo (Yakkung),DOB: February 2011,Jeon Yeo-jin,University Professor,"Busan, South Korea"
3,Chuseok Special Part 1–3 Ep 1–33,Jang Hyun-sung,Actor,Son: Jang Jun-woo (Junu),2003-07-17,Yang Huijeong,Bakery Owner,"Seoul, South Korea"
4,Chuseok Special Part 1–3 Ep 1–33,Jang Hyun-sung,Actor,Son: Jang Jun-seo,2007-07-29,Yang Huijeong,Bakery Owner,"Seoul, South Korea"
5,Ep 32–39,Do Kyung-wan,News Anchor,Son: Do Yeon-woo (Kkom-kkomyi),2014-06-13,Jang Yoon-jung,Trot Singer,"Seoul, South Korea"
6,Ep 1–58,Tablo,Rapper/Songwriter,Daughter: Lee Ha-ru,2010-05-02,Kang Hye-jung,Actress,"Seoul, South Korea"
7,Ep 59–103,Uhm Tae-woong,Actor,Daughter: Uhm Ji-on,2013-06-18,Yoon Hye-jin,Ballerina,"Opo-eup, Gwangju, Gyeonggi, South Korea"
8,"Ep 34–116, 156",Song Il-gook,Actor,Son: Song Dae-han,2012-03-16,Jung Seung-yeon,High Court Judge,"Songdo, Incheon, South Korea"
9,"Ep 34–116, 156",Song Il-gook,Actor,Son: Song Min-guk,2012-03-16,Jung Seung-yeon,High Court Judge,"Songdo, Incheon, South Korea"


In [13]:
formerNarrator_df

,episode,narrator
0,Chuseok Special Part 1–2,Yoo Ho-jeong
1,Ep 1–27,Chae Sira
2,Ep 28–37,Shin Ae-ra
3,Ep 38–59,Heo Su-gyeong
4,Ep 93,Yoona
5,Ep 96,Yoo In-na
6,Ep 97,Jang Hyun-sung
7,Ep 185–188,Kim Hyo-Jin
8,Ep 60–226,Jung Hye-young
9,Episode 227–233,Oh Sang-jin


In [14]:
specialCast_df.head()

,episode,father,fatherJob,child,childBday,mother,motherJob,filmLocation
0,Ep 184,Robert Kelly,University Professor/Political Analyst,Daughter: Marion Kelly (Ye-na),2013-03-09,Kim Jung-a,Housewife/former Yoga Instructor[13],"Busan, South Korea"
1,Ep 184,Robert Kelly,University Professor/Political Analyst,Son: James Kelly (Yu-seop),2016-06-29,Kim Jung-a,Housewife/former Yoga Instructor[13],"Busan, South Korea"
2,Ep 185–186,Park Gun-hyung,Actor,Son: Park Yi-jun,2015-06-10,Lee Chae-rim,Office Worker,"Seoul, South Korea"
3,Ep 191–192,Outsider,Rapper/Songwriter,Daughter: Shin-Yi Ro-woon,2016-03-04,Lee Young-bin,Dance Director,"Ilsan, Gyeonggi-do, South Korea"
4,Ep 130-162 (regular) Ep 194 (special appearances),In Gyo-jin,Actor,Daughter: In Ha-eun,2015-12-04,So Yi-hyun,Actress,"Jayang-dong, Gwangjin, Seoul, South Korea Namy..."


# Clean the cast tables

### episode column

In [15]:
fc_episodes_df = formerCast_df[['episode', 'father']].drop_duplicates()
cc_episodes_df = currentCast_df[['episode', 'father']].drop_duplicates()
sc_episodes_df = specialCast_df[['episode', 'father']].drop_duplicates()

In [16]:
fc_episodes_df.episode.unique()

array(['Chuseok Special Part 1–3', 'Ep 25–31',
       'Chuseok Special Part 1–3 Ep 1–33', 'Ep 32–39', 'Ep 1–58',
       'Ep 59–103', 'Ep 34–116, 156',
       'Chuseok Special Part 1–3 Ep 1–122, 155–160, 205-206',
       'Ep 127–Ep 162', 'Ep 129–Ep 162', 'Ep 117–182',
       'Ep 130-162 (regular) Ep 193 (special appearances)', 'Ep 114–210',
       'Chuseok Special Part 1–3 Ep 1–220', 'Ep. 221–259', 'Ep 268–278',
       'Ep 163–280', 'Ep 89–298'], dtype=object)

In [17]:
cc_episodes_df.episode.unique()

array(['Ep 154–Present', 'Ep 239–Present', 'Ep 289–present',
       'Ep 195, 273–274, 279, 283 (special appearances) 286–present (regular)'],
      dtype=object)

In [18]:
sc_episodes_df.episode.unique()

array(['Ep 184', 'Ep 185–186', 'Ep 191–192',
       'Ep 130-162 (regular) Ep 194 (special appearances)', 'Ep 202–204',
       'Ep 213–214', 'Ep 224–225, 239 (special appearances)',
       'Ep 227–228', 'Ep 231–232', 'Ep 235–236', 'Ep 252–253, 265',
       'Ep 264, 269, 272'], dtype=object)

In [19]:
def replace_func(x):
    present_episode = 301  # automate the present_ep
    replaceStr = (("–", "-"), 
                 ("present", str(present_episode)),
                 ("ep.", "ep"),
                 ("-ep ", "-"),
                 ("chuseok special part 1-3", "cs ep 1-3,"),
                 ("chuseok special part 1-2", "cs ep 1-3,"),
                 (")", "),"),
                 ("(regular)", ""),
                 ("episode", ""))

    for r in replaceStr:
        x = x.replace(*r)

    if len(x) != 0:
        return x.strip()
    else:
        return None


In [20]:

cc_episodes_df

,episode,father
0,Ep 154–Present,Sam Hammington
2,Ep 239–Present,Park Joo-ho
4,Ep 289–present,Moon Hee-joon
5,"Ep 195, 273–274, 279, 283 (special appearances...",Hong Kyung-min


In [21]:

def concat(text):
    text = text + " (special appearances)"
    return text

def insert(df, ctr, start_ep, end_ep, season, episodeNote, fatherOrNarrator):
    return df.append({'episode' : None , 
               fatherOrNarrator : df.loc[ctr, fatherOrNarrator],
               'startEp': start_ep,
                'endEp': end_ep,
                'season': season,
                'episodeNote': episodeNote} , ignore_index=True)

def editDataFrames(episodes_df, fatherOrNarrator):
    '''
    returns episode dataframe
    '''
    episodes_df['episode'] = (episodes_df['episode'].apply(lambda x: replace_func(x.lower())))
    column = ['startEp', 'endEp', 'season', 'episodeNote']
    for item in column:
        episodes_df[item] = None
    episodes_df = episodes_df.reset_index()
    del episodes_df['index']
    
    ctr = 0
    for row in zip(episodes_df['episode'], episodes_df[fatherOrNarrator]):
        splitEpisodes = row[0].split(', ')
        episodeNote = None
        if len(splitEpisodes) > 1:
            specialAppearance = False
            for i in range(len(splitEpisodes)):
                if "(special appearances)" in splitEpisodes[i]:
                    specialAppearance = i
            if specialAppearance:
                splitEpisodes[:specialAppearance] = map(concat, splitEpisodes[:specialAppearance])

        #print(splitEpisodes)
        for item in splitEpisodes:
            #print(type(item))
            if item == '':
                continue
            if "cs" in item:
                season = 0
                start_ep = 1
                end_ep = 3
                episodes_df = insert(episodes_df, ctr, start_ep, end_ep, season, episodeNote, fatherOrNarrator)
                continue
            if "ep" in item:
                if "(special appearances)" in item:
                    episodeNote = "special appearances"
                    item = item.replace("(special appearances)", "")
                    item = item.strip()
                eprange = re.compile(r'ep (\d+)-(\d+)')
                epNumber = eprange.search(item)
                season = 1
                if epNumber:
                    start_ep = epNumber.group(1)
                    end_ep = epNumber.group(2)
                    episodes_df = insert(episodes_df, ctr, start_ep, end_ep, season, episodeNote, fatherOrNarrator)
                    continue
                singleEp = re.compile(r'ep (\d+)')
                epNumber = singleEp.search(item)
                if epNumber:
                    start_ep = end_ep = epNumber.group(1)
                    episodes_df = insert(episodes_df, ctr, start_ep, end_ep, season, episodeNote, fatherOrNarrator)
            else:
                episodeNote = None
                if "(special appearances)" in item:
                    episodeNote = "special appearances"
                    item = item.replace("(special appearances)", "")
                    item = item.strip()
                eprange = re.compile(r'(\d+)-(\d+)')
                epNumber = eprange.search(item)
                season = 1
                if epNumber:
                    start_ep = epNumber.group(1)
                    end_ep = epNumber.group(2)
                    episodes_df = insert(episodes_df, ctr, start_ep, end_ep, season, episodeNote, fatherOrNarrator)
                    continue
                singleEp = re.compile(r'(\d+)')
                epNumber = singleEp.search(item)
                if epNumber:
                    start_ep = end_ep = epNumber.group(1)
                    episodes_df = insert(episodes_df, ctr, start_ep, end_ep, season, episodeNote, fatherOrNarrator)
            #print("ctr: ", ctr)
        ctr = ctr + 1
    return episodes_df


In [22]:
cc_episodes_df = editDataFrames(cc_episodes_df, "father")
fc_episodes_df = editDataFrames(fc_episodes_df, "father")
sc_episodes_df = editDataFrames(sc_episodes_df, "father")

In [23]:
cc_episodes_df.drop(columns='episode', inplace=True)
cc_episodes_df.dropna(axis=0, how='any', thresh=3, inplace=True)
fc_episodes_df.drop(columns='episode', inplace=True)
fc_episodes_df.dropna(axis=0, how='any', thresh=3, inplace=True)
sc_episodes_df.drop(columns='episode', inplace=True)
sc_episodes_df.dropna(axis=0, how='any', thresh=3, inplace=True)

In [24]:
cc_episodes_df

,father,startEp,endEp,season,episodeNote
4,Sam Hammington,154,301,1,None
5,Park Joo-ho,239,301,1,None
6,Moon Hee-joon,289,301,1,None
7,Hong Kyung-min,195,195,1,special appearances
8,Hong Kyung-min,273,274,1,special appearances
9,Hong Kyung-min,279,279,1,special appearances
10,Hong Kyung-min,283,283,1,special appearances
11,Hong Kyung-min,286,301,1,None


In [25]:
sc_episodes_df

,father,startEp,endEp,season,episodeNote
12,Robert Kelly,184,184,1,None
13,Park Gun-hyung,185,186,1,None
14,Outsider,191,192,1,None
15,In Gyo-jin,130,162,1,special appearances
16,In Gyo-jin,194,194,1,special appearances
17,Viktor Ahn,202,204,1,None
18,Park Hyun-bin,213,214,1,None
19,Moon Sung-min,224,225,1,special appearances
20,Moon Sung-min,239,239,1,special appearances
21,Jo Jung-chi,227,228,1,None


In [26]:
fc_episodes_df = fc_episodes_df.append(sc_episodes_df[sc_episodes_df['father'] == 'In Gyo-jin'])

In [27]:
sc_episodes_df = sc_episodes_df[~sc_episodes_df.father.str.contains("In Gyo-jin")]

In [28]:
fc_episodes_df = fc_episodes_df.drop_duplicates()
fc_episodes_df

,father,startEp,endEp,season,episodeNote
18,Lee Hyun-woo,1,3,0,None
19,Kim Jung-tae,25,31,1,None
20,Jang Hyun-sung,1,3,0,None
21,Jang Hyun-sung,1,33,1,None
22,Do Kyung-wan,32,39,1,None
23,Tablo,1,58,1,None
24,Uhm Tae-woong,59,103,1,None
25,Song Il-gook,34,116,1,None
26,Song Il-gook,156,156,1,None
27,Choo Sung-hoon,1,3,0,None


### childBday column

In [29]:
formerCast_df.childBday.unique()

array(['DOB: September 2009', 'DOB: April 2011', 'DOB: February 2011',
       '2003-07-17', '2007-07-29', '2014-06-13', '2010-05-02',
       '2013-06-18', '2012-03-16', '2011-10-24', '2013-03-28',
       '2015-10-11', '2015-12-30', '2011-03-01', '2014-02-21',
       '2015-12-04', '2015-04-12', '2013-03-15', '2015-12-01',
       '2018-05-21', '2014-07-26', '2017-09-19', '2014-10-06',
       '2007-08-14', '2013-07-18', '2014-11-14'], dtype=object)

We can see from the listed unique values of the child's birthday column that some does not follow the datetime format.

In [30]:
formerCast_df['childBday'] = formerCast_df['childBday'].apply(lambda x: x.replace('DOB: ', ""))


In [31]:
from datetime import datetime
import numpy as np

formerCast_df['note'] = np.where(formerCast_df['childBday'].apply(lambda x: np.where(re.search('[a-zA-Z]', x), True, False)),
                                "Only month and year of the child's birthday is valid. The exact date of birth isn't provided.", None)

def changeToDateFormat(x):
    if re.search('[a-zA-Z]', x):
        d = datetime.strptime(x, '%B %Y')
    else:
        d = datetime.strptime(x, '%Y-%m-%d')
    return d

formerCast_df['childBday'] = formerCast_df['childBday'].apply(lambda x: changeToDateFormat(x))

In [32]:
print(currentCast_df.childBday.unique())
print(specialCast_df.childBday.unique())
currentCast_df['childBday'] = pd.to_datetime(currentCast_df['childBday'])
specialCast_df['childBday'] = pd.to_datetime(specialCast_df['childBday'])

['2016-07-12' '2017-11-08' '2015-05-11' '2017-08-21' '2017-05-12'
 '2016-04-21' '2019-01-01']
['2013-03-09' '2016-06-29' '2015-06-10' '2016-03-04' '2015-12-04'
 '2015-12-29' '2017-05-11' '2016-02-09' '2018-01-16' '2017-02-28'
 '2017-11-20' '2016-10-03' '2018-06-16' '2015-10-25' '2014-05-20'
 '2017-11-21']


### child column

In [33]:
#specialCast_df.child.str.extract('(?P<gender>Daughter|Son):(?P<child>[A-Za-z ]*).*') #'(?P<childKor>[/(]+.*[/)]+)')
splittedChild_df = specialCast_df.child.str.extract('(?P<gender>Daughter|Son):(?P<childName>[A-Za-z -]*)[/(]*(?P<childName2>[A-Za-z -]*)[/)]*')
specialCast_df = specialCast_df.drop(['child'], axis=1)
specialCast_df = pd.concat([specialCast_df, splittedChild_df], sort=False, axis=1)

In [34]:
specialCast_df.head()

,episode,father,fatherJob,childBday,mother,motherJob,filmLocation,gender,childName,childName2
0,Ep 184,Robert Kelly,University Professor/Political Analyst,2013-03-09,Kim Jung-a,Housewife/former Yoga Instructor[13],"Busan, South Korea",Daughter,Marion Kelly,Ye-na
1,Ep 184,Robert Kelly,University Professor/Political Analyst,2016-06-29,Kim Jung-a,Housewife/former Yoga Instructor[13],"Busan, South Korea",Son,James Kelly,Yu-seop
2,Ep 185–186,Park Gun-hyung,Actor,2015-06-10,Lee Chae-rim,Office Worker,"Seoul, South Korea",Son,Park Yi-jun,
3,Ep 191–192,Outsider,Rapper/Songwriter,2016-03-04,Lee Young-bin,Dance Director,"Ilsan, Gyeonggi-do, South Korea",Daughter,Shin-Yi Ro-woon,
4,Ep 130-162 (regular) Ep 194 (special appearances),In Gyo-jin,Actor,2015-12-04,So Yi-hyun,Actress,"Jayang-dong, Gwangjin, Seoul, South Korea Namy...",Daughter,In Ha-eun,


In [35]:
splittedChild_df = currentCast_df.child.str.extract('(?P<gender>Daughter|Son):(?P<childName>[A-Za-z -]*)[/(]*(?P<childName2>[A-Za-z -]*)[/)]*')
currentCast_df = currentCast_df.drop(['child'], axis=1)
currentCast_df = pd.concat([currentCast_df, splittedChild_df], sort=False, axis=1)

In [36]:
currentCast_df.head()

,episode,father,fatherJob,childBday,mother,motherJob,filmLocation,gender,childName,childName2
0,Ep 154–Present,Sam Hammington,Television Personality,2016-07-12,Jung Yu-mi,"Housewife, Former Cafe Owner","Eunpyung-gu, Seoul, South Korea",Son,William Hammington,Jung Tae-oh
1,Ep 154–Present,Sam Hammington,Television Personality,2017-11-08,Jung Yu-mi,"Housewife, Former Cafe Owner","Eunpyung-gu, Seoul, South Korea",Son,Bentley Hammington,Jung Woo-sung
2,Ep 239–Present,Park Joo-ho,Professional Football (Soccer) player,2015-05-11,Anna Park,Housewife,"Ulsan, South Korea",Daughter,Park Na-eun,Eden
3,Ep 239–Present,Park Joo-ho,Professional Football (Soccer) player,2017-08-21,Anna Park,Housewife,"Ulsan, South Korea",Son,Park Geon-hoo,Aciel
4,Ep 289–present,Moon Hee-joon,Television Personality/Former Idol-Singer,2017-05-12,Soyul,Former Idol-singer,"Seoul, South Korea",Daughter,Moon Hee-yul,Jam-jam


In [37]:
splittedChild_df = formerCast_df.child.str.extract('(?P<gender>Daughter|Son):(?P<childName>[A-Za-z -]*)[/(]*(?P<childName2>[A-Za-z -]*)[/)]*')
formerCast_df = formerCast_df.drop(['child'], axis=1)
formerCast_df = pd.concat([formerCast_df, splittedChild_df], sort=False, axis=1)

In [38]:
formerCast_df

,episode,father,fatherJob,childBday,mother,motherJob,filmLocation,note,gender,childName,childName2
0,Chuseok Special Part 1–3,Lee Hyun-woo,Singer/Actor,2009-09-01,Yi Je-ni,Museum Curator,"Seoul, South Korea",Only month and year of the child's birthday is...,Son,Lee Dong-ha,
1,Chuseok Special Part 1–3,Lee Hyun-woo,Singer/Actor,2011-04-01,Yi Je-ni,Museum Curator,"Seoul, South Korea",Only month and year of the child's birthday is...,Son,Lee Ju-ha,
2,Ep 25–31,Kim Jung-tae,Actor,2011-02-01,Jeon Yeo-jin,University Professor,"Busan, South Korea",Only month and year of the child's birthday is...,Son,Kim Ji-hoo,Yakkung
3,Chuseok Special Part 1–3 Ep 1–33,Jang Hyun-sung,Actor,2003-07-17,Yang Huijeong,Bakery Owner,"Seoul, South Korea",None,Son,Jang Jun-woo,Junu
4,Chuseok Special Part 1–3 Ep 1–33,Jang Hyun-sung,Actor,2007-07-29,Yang Huijeong,Bakery Owner,"Seoul, South Korea",None,Son,Jang Jun-seo,
5,Ep 32–39,Do Kyung-wan,News Anchor,2014-06-13,Jang Yoon-jung,Trot Singer,"Seoul, South Korea",None,Son,Do Yeon-woo,Kkom-kkomyi
6,Ep 1–58,Tablo,Rapper/Songwriter,2010-05-02,Kang Hye-jung,Actress,"Seoul, South Korea",None,Daughter,Lee Ha-ru,
7,Ep 59–103,Uhm Tae-woong,Actor,2013-06-18,Yoon Hye-jin,Ballerina,"Opo-eup, Gwangju, Gyeonggi, South Korea",None,Daughter,Uhm Ji-on,
8,"Ep 34–116, 156",Song Il-gook,Actor,2012-03-16,Jung Seung-yeon,High Court Judge,"Songdo, Incheon, South Korea",None,Son,Song Dae-han,
9,"Ep 34–116, 156",Song Il-gook,Actor,2012-03-16,Jung Seung-yeon,High Court Judge,"Songdo, Incheon, South Korea",None,Son,Song Min-guk,


Remove the citations

In [39]:
for column in specialCast_df.columns:
    if column is "childBday":
        continue
    specialCast_df[column] = (specialCast_df[column].apply(lambda x: re.sub(r"\[.*\]", "", x)))

# re.sub(r"\[.*\]", "", specialCast_df.at[0, 'motherJob'])

### filmLocation column

In [40]:
formerCast_df['filmLocation'].unique()

array(['Seoul, South Korea', 'Busan, South Korea',
       'Opo-eup, Gwangju, Gyeonggi, South Korea',
       'Songdo, Incheon, South Korea', 'Minato, Tokyo, Japan',
       'Samseong-dong, Gangnam, Seoul, South Korea',
       'Seongsu-dong, Seongdong, Seoul, South Korea',
       'Gangnam, Seoul, South Korea',
       'Jayang-dong, Gwangjin, Seoul, South Korea Namyang-ju, Gyeonggi-do, South Korea',
       'Dongcheon-dong, South Korea (Episode 1 - 176)[11]\nGangnam, Seoul, South Korea (Episode 176–220)',
       'Bangbae-dong, Seocho-gu, Seoul, South Korea'], dtype=object)

In [41]:
currentCast_df['filmLocation'].unique()

array(['Eunpyung-gu, Seoul, South Korea', 'Ulsan, South Korea',
       'Seoul, South Korea'], dtype=object)

In [42]:
specialCast_df['filmLocation'].unique()

array(['Busan, South Korea', 'Seoul, South Korea',
       'Ilsan, Gyeonggi-do, South Korea',
       'Jayang-dong, Gwangjin, Seoul, South Korea Namyang-ju, Gyeonggi-do, South Korea',
       'Moscow, Russia', 'Hwaseong-si, Gyeonggi-do, South Korea'],
      dtype=object)

In [43]:
formerCast_df.loc[19, 'filmLocation'] = 'Dongcheon-dong, South Korea'
formerCast_df.loc[20, 'filmLocation'] = 'Gangnam, Seoul, South Korea'

In [44]:
formerCast_df

,episode,father,fatherJob,childBday,mother,motherJob,filmLocation,note,gender,childName,childName2
0,Chuseok Special Part 1–3,Lee Hyun-woo,Singer/Actor,2009-09-01,Yi Je-ni,Museum Curator,"Seoul, South Korea",Only month and year of the child's birthday is...,Son,Lee Dong-ha,
1,Chuseok Special Part 1–3,Lee Hyun-woo,Singer/Actor,2011-04-01,Yi Je-ni,Museum Curator,"Seoul, South Korea",Only month and year of the child's birthday is...,Son,Lee Ju-ha,
2,Ep 25–31,Kim Jung-tae,Actor,2011-02-01,Jeon Yeo-jin,University Professor,"Busan, South Korea",Only month and year of the child's birthday is...,Son,Kim Ji-hoo,Yakkung
3,Chuseok Special Part 1–3 Ep 1–33,Jang Hyun-sung,Actor,2003-07-17,Yang Huijeong,Bakery Owner,"Seoul, South Korea",None,Son,Jang Jun-woo,Junu
4,Chuseok Special Part 1–3 Ep 1–33,Jang Hyun-sung,Actor,2007-07-29,Yang Huijeong,Bakery Owner,"Seoul, South Korea",None,Son,Jang Jun-seo,
5,Ep 32–39,Do Kyung-wan,News Anchor,2014-06-13,Jang Yoon-jung,Trot Singer,"Seoul, South Korea",None,Son,Do Yeon-woo,Kkom-kkomyi
6,Ep 1–58,Tablo,Rapper/Songwriter,2010-05-02,Kang Hye-jung,Actress,"Seoul, South Korea",None,Daughter,Lee Ha-ru,
7,Ep 59–103,Uhm Tae-woong,Actor,2013-06-18,Yoon Hye-jin,Ballerina,"Opo-eup, Gwangju, Gyeonggi, South Korea",None,Daughter,Uhm Ji-on,
8,"Ep 34–116, 156",Song Il-gook,Actor,2012-03-16,Jung Seung-yeon,High Court Judge,"Songdo, Incheon, South Korea",None,Son,Song Dae-han,
9,"Ep 34–116, 156",Song Il-gook,Actor,2012-03-16,Jung Seung-yeon,High Court Judge,"Songdo, Incheon, South Korea",None,Son,Song Min-guk,


# narrator Tables

In [45]:
currentNarrator_df

,episode,narrator
0,"Episode 253–255, 258–present",Do Kyung-wan
1,Episode 273–present,Han Chae-ah


In [46]:
currentNarrator_df = editDataFrames(currentNarrator_df, "narrator")
currentNarrator_df

,episode,narrator,startEp,endEp,season,episodeNote
0,"253-255, 258-301",Do Kyung-wan,None,None,None,None
1,273-301,Han Chae-ah,None,None,None,None
2,None,Do Kyung-wan,253,255,1,None
3,None,Do Kyung-wan,258,301,1,None
4,None,Han Chae-ah,273,301,1,None


In [47]:
formerNarrator_df = editDataFrames(formerNarrator_df, "narrator")
formerNarrator_df

,episode,narrator,startEp,endEp,season,episodeNote
0,"cs ep 1-3,",Yoo Ho-jeong,None,None,None,None
1,ep 1-27,Chae Sira,None,None,None,None
2,ep 28-37,Shin Ae-ra,None,None,None,None
3,ep 38-59,Heo Su-gyeong,None,None,None,None
4,ep 93,Yoona,None,None,None,None
5,ep 96,Yoo In-na,None,None,None,None
6,ep 97,Jang Hyun-sung,None,None,None,None
7,ep 185-188,Kim Hyo-Jin,None,None,None,None
8,ep 60-226,Jung Hye-young,None,None,None,None
9,227-233,Oh Sang-jin,None,None,None,None


In [48]:
del currentNarrator_df['episodeNote']
del formerNarrator_df['episodeNote']
currentNarrator_df.drop(columns='episode', inplace=True)
currentNarrator_df.dropna(axis=0, how='any', thresh=3, inplace=True)
formerNarrator_df.drop(columns='episode', inplace=True)
formerNarrator_df.dropna(axis=0, how='any', thresh=3, inplace=True)

In [49]:
currentNarrator_df

,narrator,startEp,endEp,season
2,Do Kyung-wan,253,255,1
3,Do Kyung-wan,258,301,1
4,Han Chae-ah,273,301,1


In [50]:
formerNarrator_df

,narrator,startEp,endEp,season
12,Yoo Ho-jeong,1,3,0
13,Chae Sira,1,27,1
14,Shin Ae-ra,28,37,1
15,Heo Su-gyeong,38,59,1
16,Yoona,93,93,1
17,Yoo In-na,96,96,1
18,Jang Hyun-sung,97,97,1
19,Kim Hyo-Jin,185,188,1
20,Jung Hye-young,60,226,1
21,Oh Sang-jin,227,233,1


# Save to csv

In [51]:
dict_df = {'currentNarrator.csv': currentNarrator_df, 
           'formerNarrator.csv': formerNarrator_df, 
           'currentCast.csv': currentCast_df,
           'formerCast.csv': formerCast_df,
           'specialCast.csv': specialCast_df,
           'cc_episodes.csv': cc_episodes_df,
           'fc_episodes.csv': fc_episodes_df,
           'sc_episodes.csv': sc_episodes_df}

for key,val in dict_df.items():
    val.to_csv(key, encoding='utf-8', index=False)

In [52]:
#formerCast_df.groupby(['father']).first
formerCast_df.groupby(['father', 'gender'])['childName'].count()

father          gender  
Bong Tae-gyu    Daughter    1
                Son         1
Choo Sung-hoon  Daughter    1
Do Kyung-wan    Son         1
In Gyo-jin      Daughter    1
Jang Beom-june  Daughter    1
                Son         1
Jang Hyun-sung  Son         2
Ki Tae-young    Daughter    1
Kim Jung-tae    Son         1
Ko Ji-yong      Son         1
Lee Beom-soo    Daughter    1
                Son         1
Lee Dong-gook   Daughter    4
                Son         1
Lee Hwi-jae     Son         2
Lee Hyun-woo    Son         2
Oh Ji-ho        Daughter    1
Song Il-gook    Son         3
Tablo           Daughter    1
Uhm Tae-woong   Daughter    1
Yang Dong-geun  Daughter    1
                Son         1
Name: childName, dtype: int64

In [53]:
currentCast_df.groupby(['father', 'gender'])['childName'].count()

father          gender  
Hong Kyung-min  Daughter    2
Moon Hee-joon   Daughter    1
Park Joo-ho     Daughter    1
                Son         1
Sam Hammington  Son         2
Name: childName, dtype: int64

In [67]:
from geopy.geocoders import Nominatim

def get_latitude(location):
    print(location)
    try: return geolocator.geocode(location).latitude
    except: return None
        
def get_longitude(location):
    print(location)
    try: return geolocator.geocode(location).longitude
    except: return None
    
def get_address(location):
    print(location)
    try: return geolocator.geocode(location).address
    except: return None

In [68]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("175 5th Avenue NYC")
print(location.address)
print((location.latitude, location.longitude))
(40.7410861, -73.9896297241625)
print(location.raw)


Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, New York, 10010, United States of America
(40.7410861, -73.9896298241625)
{'place_id': 138642704, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 264768896, 'boundingbox': ['40.7407597', '40.7413004', '-73.9898715', '-73.9895014'], 'lat': '40.7410861', 'lon': '-73.9896298241625', 'display_name': 'Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, New York, 10010, United States of America', 'class': 'tourism', 'type': 'attraction', 'importance': 0.6830033155219741, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_point_of_interest.p.20.png'}


### Get Latitude, Longitude, and Address of filmLocation

In [71]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")

for item in formerCast_df['filmLocation']:
    if item:
        location = geolocator.geocode(item)
        if location:
            print(location.address)
            print((location.latitude, location.longitude))
        else:
            print("The item with no address: ", item)

서울, 대한민국
(37.5666791, 126.9782914)
서울, 대한민국
(37.5666791, 126.9782914)
부산, 대한민국
(35.1799528, 129.0752365)
서울, 대한민국
(37.5666791, 126.9782914)
서울, 대한민국
(37.5666791, 126.9782914)
서울, 대한민국
(37.5666791, 126.9782914)
서울, 대한민국
(37.5666791, 126.9782914)
오포읍, 광주시, 경기도, 12774, 대한민국
(37.36212, 127.21115)
송도동, 연수구, 인천, 대한민국
(37.3851499, 126.6457433)
송도동, 연수구, 인천, 대한민국
(37.3851499, 126.6457433)
송도동, 연수구, 인천, 대한민국
(37.3851499, 126.6457433)
港区, 東京都, 関東地方, 日本 (Japan)
(35.6432274, 139.7400553)
삼성동, 강남구, 서울, 대한민국
(37.5147838, 127.0543618)
삼성동, 강남구, 서울, 대한민국
(37.5147838, 127.0543618)
성수, 100, 아차산로, 성수2가3동, 성동구, 서울, 04793, 대한민국
(37.5445436, 127.0561216)
강남, 강남대로, 역삼1동, 강남구, 서울, 06615, 대한민국
(37.4979002, 127.0276077)
강남, 강남대로, 역삼1동, 강남구, 서울, 06615, 대한민국
(37.4979002, 127.0276077)
The item with no address:  Jayang-dong, Gwangjin, Seoul, South Korea Namyang-ju, Gyeonggi-do, South Korea
서울, 대한민국
(37.5666791, 126.9782914)
동천동, 경주시, 경상북도, 38099, 대한민국
(35.85158, 129.23434)
강남, 강남대로, 역삼1동, 강남구, 서울, 06615, 대한민국
(37.4

In [73]:
formerCast_df['latitude'] = formerCast_df.filmLocation.apply(get_latitude)
formerCast_df['longitude'] = formerCast_df.filmLocation.apply(get_longitude)
formerCast_df['address'] = formerCast_df.filmLocation.apply(get_address)

Seoul, South Korea
Seoul, South Korea
Busan, South Korea
Seoul, South Korea
Seoul, South Korea
Seoul, South Korea
Seoul, South Korea
Opo-eup, Gwangju, Gyeonggi, South Korea
Songdo, Incheon, South Korea
Songdo, Incheon, South Korea
Songdo, Incheon, South Korea
Minato, Tokyo, Japan
Samseong-dong, Gangnam, Seoul, South Korea
Samseong-dong, Gangnam, Seoul, South Korea
Seongsu-dong, Seongdong, Seoul, South Korea
Gangnam, Seoul, South Korea
Gangnam, Seoul, South Korea
Jayang-dong, Gwangjin, Seoul, South Korea Namyang-ju, Gyeonggi-do, South Korea
Seoul, South Korea
Dongcheon-dong, South Korea
Gangnam, Seoul, South Korea
Seoul, South Korea
Seoul, South Korea
Seoul, South Korea
Seoul, South Korea
Bangbae-dong, Seocho-gu, Seoul, South Korea
Songdo, Incheon, South Korea
Songdo, Incheon, South Korea
Songdo, Incheon, South Korea
Songdo, Incheon, South Korea
Songdo, Incheon, South Korea
Seoul, South Korea
Seoul, South Korea
Busan, South Korea
Seoul, South Korea
Seoul, South Korea
Seoul, South Korea
